# 第 3 阶段：LangGraph 工作流 + 人机交互

本 Notebook 用于学习和调试第 3 阶段的核心功能：
- LangGraph 有状态工作流
- 学习工作流（规划、检索、出题、评分、反馈）
- 检查点（Checkpoint）机制
- 人机交互（Human-in-the-Loop）
- 工作流状态管理
- 流式输出工作流进度

**参考文档**: `docs/stage_03/`  
**参考脚本**: `scripts/test_workflow.py`

## 1. 环境准备

In [1]:
# 添加项目根目录到 Python 路径
import sys
from pathlib import Path

backend_dir = Path.cwd()
if backend_dir.name == 'notebooks':
    backend_dir = backend_dir.parent

if str(backend_dir) not in sys.path:
    sys.path.insert(0, str(backend_dir))

print(f"✅ 项目根目录: {backend_dir}")

# 导入必要的模块
import uuid
from datetime import datetime
from config import get_logger

logger = get_logger(__name__)

2025-12-06 22:17:37.611 | INFO     | config.logging:setup_logging:83 | 📝 日志系统初始化完成 - 级别: INFO, 文件: logs/app.log


✅ 项目根目录: /Users/yuan/dev/ai-projects/lc-studylab/backend


## 2. 工作流基础概念

### 2.1 学习工作流节点

学习工作流包含以下节点：

```
开始 → 规划学习路径 → 知识检索 → 出题测试 → [等待用户答题] → 自动评分 → 反馈建议 → 结束
```

- **规划节点**: 分析学习主题，制定学习计划
- **检索节点**: 从知识库检索相关资料
- **出题节点**: 根据主题生成测试题
- **评分节点**: 自动评分用户答案
- **反馈节点**: 分析学习效果，提供改进建议

## 3. 启动学习工作流

In [2]:
from workflows.study_flow_graph import start_study_flow

# 生成唯一的 thread_id
thread_id = f"notebook_test_{uuid.uuid4().hex[:8]}"
print(f"🆔 Thread ID: {thread_id}\n")

# 定义学习问题
user_question = "我想学习 Python 的基础知识，包括变量、数据类型和控制流"

print(f"📝 学习问题: {user_question}\n")
print("🔄 启动工作流...\n")

🆔 Thread ID: notebook_test_b7bd2588

📝 学习问题: 我想学习 Python 的基础知识，包括变量、数据类型和控制流

🔄 启动工作流...



In [3]:
# 启动工作流
result = start_study_flow(
    user_question=user_question,
    thread_id=thread_id
)

print("✅ 工作流启动成功！\n")
print(f"{'='*60}")
print("工作流状态")
print(f"{'='*60}")
print(f"当前步骤: {result.get('current_step')}")
print(f"更新时间: {result.get('updated_at')}")
print(f"{'='*60}")

2025-12-06 22:17:46.008 | INFO     | workflows.study_flow_graph:start_study_flow:217 | [Study Flow] 启动新的学习工作流，thread_id=notebook_test_b7bd2588
2025-12-06 22:17:46.009 | INFO     | workflows.study_flow_graph:get_study_flow_app:198 | [Study Flow Graph] 初始化全局工作流实例
2025-12-06 22:17:46.010 | INFO     | workflows.study_flow_graph:create_study_flow_graph:91 | [Study Flow Graph] 开始创建学习工作流图
2025-12-06 22:17:46.011 | INFO     | workflows.study_flow_graph:create_study_flow_graph:97 | [Study Flow Graph] 添加工作流节点...
2025-12-06 22:17:46.013 | INFO     | workflows.study_flow_graph:create_study_flow_graph:127 | [Study Flow Graph] 添加工作流边...
2025-12-06 22:17:46.014 | INFO     | workflows.study_flow_graph:create_study_flow_graph:161 | [Study Flow Graph] 配置检查点存储: data/checkpoints/study_flow.db
2025-12-06 22:17:46.014 | INFO     | workflows.study_flow_graph:create_study_flow_graph:169 | [Study Flow Graph] 编译工作流图...
2025-12-06 22:17:46.029 | INFO     | workflows.study_flow_graph:create_study_flow_graph:177 |

✅ 工作流启动成功！

工作流状态
当前步骤: quiz_generated
更新时间: 2025-12-06T22:18:43.383701


## 4. 查看学习计划

In [4]:
# 查看生成的学习计划
if result.get('learning_plan'):
    plan = result['learning_plan']
    
    print("\n📚 学习计划")
    print(f"{'='*60}")
    print(f"主题: {plan.get('topic')}")
    print(f"难度: {plan.get('difficulty')}")
    print(f"预计时间: {plan.get('estimated_time')} 分钟")
    
    print(f"\n学习目标 ({len(plan.get('objectives', []))})个):")
    for i, obj in enumerate(plan.get('objectives', []), 1):
        print(f"  {i}. {obj}")
    
    print(f"\n关键知识点 ({len(plan.get('key_points', []))})个):")
    for i, point in enumerate(plan.get('key_points', []), 1):
        print(f"  {i}. {point}")
    
    print(f"{'='*60}")
else:
    print("⚠️  未生成学习计划")


📚 学习计划
主题: Python 基础语法：变量、数据类型和控制流
难度: beginner
预计时间: 180 分钟

学习目标 (3)个):
  1. 能够定义并使用变量存储不同类型的值
  2. 熟练掌握 Python 中的基本数据类型及其操作方法
  3. 能够使用条件语句控制程序的执行流程

关键知识点 (5)个):
  1. 变量的定义与使用
  2. 基本数据类型（整数、浮点数、字符串、布尔值）
  3. 数据类型的类型转换
  4. 条件语句（if-elif-else）
  5. 循环语句（for 和 while）


## 5. 查看练习题

In [5]:
# 查看生成的练习题
if result.get('quiz'):
    quiz = result['quiz']
    questions = quiz.get('questions', [])
    
    print("\n📝 练习题")
    print(f"{'='*60}")
    print(f"题目数量: {len(questions)}")
    print(f"总分: {quiz.get('total_points')} 分")
    print(f"建议用时: {quiz.get('time_limit')} 分钟")
    print(f"{'='*60}\n")
    
    # 显示每道题
    for i, q in enumerate(questions, 1):
        print(f"\n第 {i} 题 ({q['points']} 分) - {q['type']}")
        print(f"{'─'*60}")
        print(f"问题: {q['question']}")
        
        if q.get('options'):
            print("\n选项:")
            for opt in q['options']:
                print(f"  {opt}")
        
        # 显示正确答案（仅用于调试）
        print(f"\n✅ 正确答案: {q.get('answer')}")
        print(f"💡 解析: {q.get('explanation', '无')}")
else:
    print("⚠️  未生成练习题")


📝 练习题
题目数量: 8
总分: 100 分
建议用时: 30 分钟


第 1 题 (15 分) - multiple_choice
────────────────────────────────────────────────────────────
问题: 关于Python变量的定义与使用，以下哪项是正确的？

选项:
  变量是程序中用来存储数据的容器。
  变量必须在使用前声明类型。
  变量名可以以数字开头。
  变量存储的是数据的实际值。

✅ 正确答案: 变量的定义是通过赋值操作完成的，例如：x = 5。变量名应符合命名规则（如不能以数字开头，不能使用关键字等），变量存储的是数据的引用，而不是数据本身。
💡 解析: 变量是程序中用来存储数据的容器，通过赋值操作（如 x = 5）来定义。变量名需要符合命名规范，且变量存储的是数据的引用，而非数据值本身。

第 2 题 (15 分) - multiple_choice
────────────────────────────────────────────────────────────
问题: 关于Python的基本数据类型，以下哪项是正确的？

选项:
  整数可以表示小数部分。
  浮点数不能进行数学运算。
  整数和浮点数是Python中完全相同的类型。
  整数和浮点数是两种不同的基本数据类型。

✅ 正确答案: 整数（int）和浮点数（float）是Python中两种不同的数据类型，整数用于表示无小数部分的数字，浮点数用于表示带有小数部分的数字。
💡 解析: Python中的整数类型（int）用于表示整数值，如123；浮点数类型（float）用于表示带小数点的数值，如3.14。它们是两种不同的数据类型，且在运算中可能有精度差异。

第 3 题 (15 分) - multiple_choice
────────────────────────────────────────────────────────────
问题: 关于Python中的类型转换，以下哪项描述是正确的？

选项:
  类型转换只能在字符串和整数之间进行。
  类型转换不需要使用函数。
  类型转换是将数据从一种类型转换为另一种类型的过程。
  类型转换不会影响数据的数值。

✅ 正确答案: 类型转换是将一个数据类型的值

## 6. 提交答案

### 6.1 准备答案

In [6]:
# 准备用户答案
# 这里我们模拟用户答题，你可以修改答案来测试不同的评分结果

quiz = result.get('quiz', {})
questions = quiz.get('questions', [])

user_answers = {}

print("准备答案...\n")

for i, q in enumerate(questions):
    q_id = q['id']
    q_type = q['type']
    
    # 这里我们故意让第一题答对，其他题答错，来测试评分功能
    if i == 0:
        # 第一题答对
        user_answers[q_id] = q['answer']
        print(f"第 {i+1} 题: 答对 ✅")
    else:
        # 其他题答错
        if q_type == 'multiple_choice':
            wrong_options = ['A', 'B', 'C', 'D']
            if q['answer'] in wrong_options:
                wrong_options.remove(q['answer'])
            user_answers[q_id] = wrong_options[0]
            print(f"第 {i+1} 题: 答错 ❌ (选择了 {wrong_options[0]})")
        else:
            user_answers[q_id] = "错误答案"
            print(f"第 {i+1} 题: 答错 ❌")

print(f"\n共准备 {len(user_answers)} 个答案")

准备答案...

第 1 题: 答对 ✅
第 2 题: 答错 ❌ (选择了 A)
第 3 题: 答错 ❌ (选择了 A)
第 4 题: 答错 ❌ (选择了 A)
第 5 题: 答错 ❌ (选择了 A)
第 6 题: 答错 ❌
第 7 题: 答错 ❌
第 8 题: 答错 ❌

共准备 8 个答案


### 6.2 提交答案并评分

In [7]:
from workflows.study_flow_graph import submit_answers

print("✍️  提交答案...\n")

# 提交答案
result = submit_answers(
    thread_id=thread_id,
    user_answers=user_answers
)

print("✅ 答案提交成功！\n")

2025-12-06 22:54:03.224 | INFO     | workflows.study_flow_graph:submit_answers:269 | [Study Flow] 提交答案，thread_id=notebook_test_b7bd2588
2025-12-06 22:54:03.225 | INFO     | workflows.study_flow_graph:submit_answers:282 | [Study Flow] 当前状态: quiz_generated
2025-12-06 22:54:03.227 | INFO     | workflows.study_flow_graph:submit_answers:294 | [Study Flow] 继续执行工作流...
2025-12-06 22:54:03.228 | INFO     | workflows.study_flow_graph:human_review_node:112 | [Human Review Node] 等待用户提交答案...
2025-12-06 22:54:03.229 | INFO     | workflows.nodes.grading_node:grading_node:34 | [Grading Node] 开始评分
2025-12-06 22:54:03.230 | INFO     | core.models:get_chat_model:81 | 🤖 创建聊天模型: Qwen/Qwen3-8B (temperature=0.7, streaming=True)
2025-12-06 22:54:03.231 | INFO     | workflows.nodes.grading_node:grading_node:79 | [Grading Node] 使用 LLM 评分简答题: q8


✍️  提交答案...



2025-12-06 22:54:04.969 | INFO     | workflows.nodes.grading_node:grading_node:149 | [Grading Node] 评分完成: 15分 (1/8 题正确)
2025-12-06 22:54:04.972 | INFO     | workflows.nodes.feedback_node:feedback_node:33 | [Feedback Node] 开始生成反馈
2025-12-06 22:54:04.973 | INFO     | workflows.nodes.feedback_node:feedback_node:41 | [Feedback Node] 当前得分: 15, 重试次数: 0
2025-12-06 22:54:04.973 | INFO     | core.models:get_chat_model:81 | 🤖 创建聊天模型: Qwen/Qwen3-8B (temperature=0.7, streaming=True)
2025-12-06 22:54:04.975 | INFO     | workflows.nodes.feedback_node:feedback_node:76 | [Feedback Node] 调用 LLM 生成个性化反馈...
2025-12-06 22:54:07.862 | INFO     | workflows.nodes.feedback_node:feedback_node:80 | [Feedback Node] 反馈生成完成
2025-12-06 22:54:07.863 | INFO     | workflows.nodes.feedback_node:feedback_node:100 | [Feedback Node] should_retry=True, new_retry_count=1
2025-12-06 22:54:07.864 | INFO     | workflows.study_flow_graph:should_continue:52 | [Conditional Edge] should_retry=True, retry_count=1
2025-12-06 22:54:0

✅ 答案提交成功！



## 7. 查看评分结果

In [8]:
# 查看评分结果
if result.get('score') is not None:
    score = result['score']
    score_details = result.get('score_details', {})
    
    print("\n📊 评分结果")
    print(f"{'='*60}")
    print(f"总分: {score} 分")
    print(f"答对: {score_details.get('correct_count', 0)}/{score_details.get('total_count', 0)} 题")
    print(f"正确率: {score_details.get('correct_count', 0) / score_details.get('total_count', 1) * 100:.1f}%")
    print(f"是否需要重试: {'是' if result.get('should_retry') else '否'}")
    print(f"{'='*60}")
    
    # 显示每题的评分详情
    if score_details.get('question_scores'):
        print("\n详细评分:")
        for q_score in score_details['question_scores']:
            status = "✅" if q_score.get('correct') else "❌"
            print(f"  {status} 题目 {q_score.get('question_id')}: {q_score.get('score', 0)} 分")
            if q_score.get('comment'):
                print(f"     评语: {q_score['comment']}")
else:
    print("⚠️  未获取到评分结果")


📊 评分结果
总分: 15 分
答对: 1/8 题
正确率: 12.5%
是否需要重试: 是

详细评分:
  ❌ 题目 q1: 0 分
  ❌ 题目 q2: 0 分
  ❌ 题目 q3: 0 分
  ❌ 题目 q4: 0 分
  ❌ 题目 q5: 0 分
  ❌ 题目 q6: 0 分
  ❌ 题目 q7: 0 分
  ❌ 题目 q8: 0 分


## 8. 查看反馈建议

In [9]:
# 查看反馈
if result.get('feedback'):
    print("\n💬 学习反馈")
    print(f"{'='*60}")
    print(result['feedback'])
    print(f"{'='*60}")
else:
    print("⚠️  未获取到反馈")


💬 学习反馈
你的整体表现有进步空间，但已经迈出了学习Python的第一步，继续保持热情！  
针对错题，建议你加强对基础数据类型和控制流语句的理解，尤其是变量类型转换、条件语句和循环结构。多复习相关概念，结合实例练习，能更好地掌握这些知识。  
下一步可以重点巩固变量与数据类型，熟悉常见类型转换方法，并尝试编写简单的条件与循环程序。  
相信你通过努力一定能够掌握这些基础内容，加油！你一定可以做到的！


## 9. 工作流状态管理

### 9.1 查询当前状态

In [10]:
from workflows.study_flow_graph import get_workflow_state

# 查询工作流状态
state = get_workflow_state(thread_id)

if state:
    print("\n📊 工作流状态")
    print(f"{'='*60}")
    print(f"Thread ID: {thread_id}")
    print(f"当前步骤: {state.get('current_step')}")
    print(f"更新时间: {state.get('updated_at')}")
    print(f"是否完成: {'是' if state.get('is_finished') else '否'}")
    
    if state.get('error'):
        print(f"错误: {state['error']}")
    
    print(f"{'='*60}")
    
    # 显示状态中的关键数据
    print("\n状态数据:")
    print(f"  - 有学习计划: {'是' if state.get('learning_plan') else '否'}")
    print(f"  - 有练习题: {'是' if state.get('quiz') else '否'}")
    print(f"  - 有用户答案: {'是' if state.get('user_answers') else '否'}")
    print(f"  - 有评分: {'是' if state.get('score') is not None else '否'}")
    print(f"  - 有反馈: {'是' if state.get('feedback') else '否'}")
else:
    print("⚠️  无法获取工作流状态")

2025-12-06 23:04:19.510 | INFO     | workflows.study_flow_graph:get_workflow_state:312 | [Study Flow] 获取工作流状态，thread_id=notebook_test_b7bd2588



📊 工作流状态
Thread ID: notebook_test_b7bd2588
当前步骤: quiz_generated
更新时间: 2025-12-06T22:55:14.245797
是否完成: 否

状态数据:
  - 有学习计划: 是
  - 有练习题: 是
  - 有用户答案: 是
  - 有评分: 是
  - 有反馈: 是


### 9.2 查看工作流历史

In [11]:
from workflows.study_flow_graph import get_workflow_history

# 查看工作流历史
history = get_workflow_history(thread_id)

print(f"\n📜 工作流历史 (共 {len(history)} 个检查点)")
print(f"{'='*60}\n")

for i, h in enumerate(history, 1):
    print(f"{i}. {h.get('step', 'unknown')}")
    print(f"   时间: {h.get('timestamp', 'unknown')}")
    print(f"   配置: {h.get('config', {})}")
    print()

2025-12-06 23:04:50.202 | INFO     | workflows.study_flow_graph:get_workflow_history:337 | [Study Flow] 获取工作流历史，thread_id=notebook_test_b7bd2588



📜 工作流历史 (共 10 个检查点)

1. quiz_generated
   时间: 2025-12-06T22:55:14.245797
   配置: {}

2. feedback
   时间: 2025-12-06T22:54:07.864447
   配置: {}

3. grading
   时间: 2025-12-06T22:54:04.970413
   配置: {}

4. waiting_for_answers
   时间: 2025-12-06T22:54:03.229043
   配置: {}

5. quiz_generated
   时间: 2025-12-06T22:54:03.226736
   配置: {}

6. quiz_generated
   时间: 2025-12-06T22:18:43.383701
   配置: {}

7. retrieval
   时间: 2025-12-06T22:17:52.115191
   配置: {}

8. planner
   时间: 2025-12-06T22:17:51.962790
   配置: {}

9. start
   时间: 2025-12-06T22:17:46.031114
   配置: {}

10. None
   时间: None
   配置: {}



## 10. 检查点恢复测试

### 10.1 创建新的工作流并暂停

In [12]:
# 创建新的工作流用于测试检查点恢复
recovery_thread_id = f"recovery_test_{uuid.uuid4().hex[:8]}"

print(f"🆔 新 Thread ID: {recovery_thread_id}\n")
print("启动工作流（将在答题环节暂停）...\n")

recovery_result = start_study_flow(
    user_question="学习机器学习的基础概念",
    thread_id=recovery_thread_id
)

print("✅ 工作流已暂停在答题环节")
print(f"当前步骤: {recovery_result.get('current_step')}")

2025-12-06 23:05:42.084 | INFO     | workflows.study_flow_graph:start_study_flow:217 | [Study Flow] 启动新的学习工作流，thread_id=recovery_test_d426ca83
2025-12-06 23:05:42.085 | INFO     | workflows.study_flow_graph:start_study_flow:250 | [Study Flow] 开始执行工作流...
2025-12-06 23:05:42.087 | INFO     | workflows.nodes.planner_node:planner_node:44 | [Planner Node] 开始生成学习计划，用户问题: 学习机器学习的基础概念
2025-12-06 23:05:42.088 | INFO     | core.models:get_chat_model:81 | 🤖 创建聊天模型: Qwen/Qwen3-8B (temperature=0.7, streaming=True)
2025-12-06 23:05:42.089 | INFO     | workflows.nodes.planner_node:planner_node:68 | [Planner Node] 调用 LLM 生成学习计划...


🆔 新 Thread ID: recovery_test_d426ca83

启动工作流（将在答题环节暂停）...



2025-12-06 23:05:53.490 | INFO     | workflows.nodes.planner_node:planner_node:83 | [Planner Node] 学习计划生成成功: 机器学习基础概念
2025-12-06 23:05:53.491 | INFO     | workflows.nodes.retrieval_node:retrieval_node:35 | [Retrieval Node] 开始检索相关文档
2025-12-06 23:05:53.492 | INFO     | workflows.nodes.retrieval_node:retrieval_node:49 | [Retrieval Node] 主查询: 机器学习基础概念
2025-12-06 23:05:53.493 | INFO     | rag.index_manager:__init__:79 | 📁 索引管理器初始化: data/indexes
2025-12-06 23:05:53.493 | INFO     | rag.embeddings:get_embeddings:162 | 🔢 创建 Embedding 模型: Qwen/Qwen3-Embedding-8B
2025-12-06 23:05:53.596 | WARNING  | workflows.nodes.retrieval_node:retrieval_node:64 | [Retrieval Node] 索引不存在，返回空结果
2025-12-06 23:05:53.597 | INFO     | workflows.nodes.retrieval_node:retrieval_node:77 | [Retrieval Node] 检索到 0 个相关文档
2025-12-06 23:05:53.597 | INFO     | workflows.nodes.retrieval_node:retrieval_node:96 | [Retrieval Node] 最终检索到 0 个文档
2025-12-06 23:05:53.598 | INFO     | workflows.nodes.quiz_generator_node:quiz_generator_

✅ 工作流已暂停在答题环节
当前步骤: quiz_generated


### 10.2 模拟程序重启，从检查点恢复

In [13]:
print("\n🔄 模拟程序重启，从检查点恢复状态...\n")

# 从检查点恢复状态
recovered_state = get_workflow_state(recovery_thread_id)

if recovered_state:
    print("✅ 成功从检查点恢复状态！\n")
    
    print("恢复的状态:")
    print(f"  步骤: {recovered_state.get('current_step')}")
    print(f"  更新时间: {recovered_state.get('updated_at')}")
    
    # 验证数据完整性
    data_integrity = []
    
    if recovered_state.get('quiz'):
        data_integrity.append("✅ 练习题数据完整")
    else:
        data_integrity.append("❌ 练习题数据丢失")
    
    if recovered_state.get('learning_plan'):
        data_integrity.append("✅ 学习计划数据完整")
    else:
        data_integrity.append("❌ 学习计划数据丢失")
    
    print("\n数据完整性检查:")
    for check in data_integrity:
        print(f"  {check}")
    
    print("\n✅ 检查点恢复测试完成！")
else:
    print("❌ 无法恢复状态")

2025-12-06 23:16:18.011 | INFO     | workflows.study_flow_graph:get_workflow_state:312 | [Study Flow] 获取工作流状态，thread_id=recovery_test_d426ca83



🔄 模拟程序重启，从检查点恢复状态...

✅ 成功从检查点恢复状态！

恢复的状态:
  步骤: quiz_generated
  更新时间: 2025-12-06T23:07:59.678209

数据完整性检查:
  ✅ 练习题数据完整
  ✅ 学习计划数据完整

✅ 检查点恢复测试完成！


## 11. 重试机制测试

In [14]:
# 测试重试机制（得分低于60分时触发）
retry_thread_id = f"retry_test_{uuid.uuid4().hex[:8]}"

print(f"🆔 Thread ID: {retry_thread_id}\n")
print("启动工作流...\n")

retry_result = start_study_flow(
    user_question="学习深度学习的基本概念",
    thread_id=retry_thread_id
)

print("✅ 工作流启动成功\n")

# 故意全部答错
print("✍️  提交全错答案（测试重试机制）...\n")

quiz = retry_result.get('quiz', {})
questions = quiz.get('questions', [])

# 全部答错
wrong_answers = {}
for q in questions:
    q_id = q['id']
    q_type = q['type']
    
    if q_type == 'multiple_choice':
        wrong_options = ['A', 'B', 'C', 'D']
        if q['answer'] in wrong_options:
            wrong_options.remove(q['answer'])
        wrong_answers[q_id] = wrong_options[0]
    else:
        wrong_answers[q_id] = "错误答案"

retry_result = submit_answers(retry_thread_id, wrong_answers)

score = retry_result.get('score', 0)
should_retry = retry_result.get('should_retry', False)

print(f"\n📊 评分结果: {score} 分")
print(f"🔄 是否触发重试: {'是' if should_retry else '否'}")

if should_retry:
    print("\n✅ 重试机制正常工作！")
    print("   系统已自动生成新的练习题")
    
    # 查看新题目
    state = get_workflow_state(retry_thread_id)
    if state.get('quiz'):
        new_quiz = state['quiz']
        print(f"   新练习题: {len(new_quiz.get('questions', []))} 题")
else:
    print("\n⚠️  未触发重试（可能得分高于60分）")

2025-12-06 23:17:25.846 | INFO     | workflows.study_flow_graph:start_study_flow:217 | [Study Flow] 启动新的学习工作流，thread_id=retry_test_247073cd
2025-12-06 23:17:25.847 | INFO     | workflows.study_flow_graph:start_study_flow:250 | [Study Flow] 开始执行工作流...
2025-12-06 23:17:25.849 | INFO     | workflows.nodes.planner_node:planner_node:44 | [Planner Node] 开始生成学习计划，用户问题: 学习深度学习的基本概念
2025-12-06 23:17:25.850 | INFO     | core.models:get_chat_model:81 | 🤖 创建聊天模型: Qwen/Qwen3-8B (temperature=0.7, streaming=True)
2025-12-06 23:17:25.851 | INFO     | workflows.nodes.planner_node:planner_node:68 | [Planner Node] 调用 LLM 生成学习计划...


🆔 Thread ID: retry_test_247073cd

启动工作流...



2025-12-06 23:17:34.709 | INFO     | workflows.nodes.planner_node:planner_node:83 | [Planner Node] 学习计划生成成功: 深度学习的基本概念
2025-12-06 23:17:34.711 | INFO     | workflows.nodes.retrieval_node:retrieval_node:35 | [Retrieval Node] 开始检索相关文档
2025-12-06 23:17:34.712 | INFO     | workflows.nodes.retrieval_node:retrieval_node:49 | [Retrieval Node] 主查询: 深度学习的基本概念
2025-12-06 23:17:34.713 | INFO     | rag.index_manager:__init__:79 | 📁 索引管理器初始化: data/indexes
2025-12-06 23:17:34.714 | INFO     | rag.embeddings:get_embeddings:162 | 🔢 创建 Embedding 模型: Qwen/Qwen3-Embedding-8B
2025-12-06 23:17:34.816 | WARNING  | workflows.nodes.retrieval_node:retrieval_node:64 | [Retrieval Node] 索引不存在，返回空结果
2025-12-06 23:17:34.816 | INFO     | workflows.nodes.retrieval_node:retrieval_node:77 | [Retrieval Node] 检索到 0 个相关文档
2025-12-06 23:17:34.816 | INFO     | workflows.nodes.retrieval_node:retrieval_node:96 | [Retrieval Node] 最终检索到 0 个文档
2025-12-06 23:17:34.817 | INFO     | workflows.nodes.quiz_generator_node:quiz_generato

✅ 工作流启动成功

✍️  提交全错答案（测试重试机制）...



2025-12-06 23:18:53.014 | INFO     | workflows.nodes.grading_node:grading_node:149 | [Grading Node] 评分完成: 0分 (0/7 题正确)
2025-12-06 23:18:53.016 | INFO     | workflows.nodes.feedback_node:feedback_node:33 | [Feedback Node] 开始生成反馈
2025-12-06 23:18:53.016 | INFO     | workflows.nodes.feedback_node:feedback_node:41 | [Feedback Node] 当前得分: 0, 重试次数: 0
2025-12-06 23:18:53.017 | INFO     | core.models:get_chat_model:81 | 🤖 创建聊天模型: Qwen/Qwen3-8B (temperature=0.7, streaming=True)
2025-12-06 23:18:53.017 | INFO     | workflows.nodes.feedback_node:feedback_node:76 | [Feedback Node] 调用 LLM 生成个性化反馈...
2025-12-06 23:18:57.629 | INFO     | workflows.nodes.feedback_node:feedback_node:80 | [Feedback Node] 反馈生成完成
2025-12-06 23:18:57.630 | INFO     | workflows.nodes.feedback_node:feedback_node:100 | [Feedback Node] should_retry=True, new_retry_count=1
2025-12-06 23:18:57.631 | INFO     | workflows.study_flow_graph:should_continue:52 | [Conditional Edge] should_retry=True, retry_count=1
2025-12-06 23:18:57.


📊 评分结果: 0 分
🔄 是否触发重试: 是

✅ 重试机制正常工作！
   系统已自动生成新的练习题
   新练习题: 12 题


## 12. 流式输出工作流进度

In [16]:
# 演示流式输出工作流进度
from workflows.study_flow_graph import create_study_flow_graph

stream_thread_id = f"stream_test_{uuid.uuid4().hex[:8]}"

print(f"🆔 Thread ID: {stream_thread_id}\n")
print("📡 流式输出工作流进度:\n")
print(f"{'='*60}\n")

# 创建工作流
workflow = create_study_flow_graph()

# 配置
config = {"configurable": {"thread_id": stream_thread_id}}

# 输入
inputs = {
    "user_question": "学习 Python 函数的基础知识",
    "user_id": "test_user"
}

# 流式执行
for event in workflow.stream(inputs, config):
    print(f"事件: {event}")
    print()

print(f"{'='*60}")
print("\n✅ 工作流执行完成（已暂停在答题环节）")

2025-12-06 23:23:33.244 | INFO     | workflows.study_flow_graph:create_study_flow_graph:91 | [Study Flow Graph] 开始创建学习工作流图
2025-12-06 23:23:33.245 | INFO     | workflows.study_flow_graph:create_study_flow_graph:97 | [Study Flow Graph] 添加工作流节点...
2025-12-06 23:23:33.246 | INFO     | workflows.study_flow_graph:create_study_flow_graph:127 | [Study Flow Graph] 添加工作流边...
2025-12-06 23:23:33.247 | INFO     | workflows.study_flow_graph:create_study_flow_graph:161 | [Study Flow Graph] 配置检查点存储: data/checkpoints/study_flow.db
2025-12-06 23:23:33.247 | INFO     | workflows.study_flow_graph:create_study_flow_graph:169 | [Study Flow Graph] 编译工作流图...
2025-12-06 23:23:33.259 | INFO     | workflows.study_flow_graph:create_study_flow_graph:177 | [Study Flow Graph] 学习工作流图创建完成
2025-12-06 23:23:33.259 | INFO     | workflows.study_flow_graph:create_study_flow_graph:178 | [Study Flow Graph] 中断点设置在: human_review (等待用户答题)
2025-12-06 23:23:33.261 | INFO     | workflows.nodes.planner_node:planner_node:44 | [Pla

🆔 Thread ID: stream_test_e1fb4518

📡 流式输出工作流进度:




2025-12-06 23:23:40.668 | INFO     | workflows.nodes.planner_node:planner_node:83 | [Planner Node] 学习计划生成成功: Python 函数基础
2025-12-06 23:23:40.670 | INFO     | workflows.nodes.retrieval_node:retrieval_node:35 | [Retrieval Node] 开始检索相关文档
2025-12-06 23:23:40.671 | INFO     | workflows.nodes.retrieval_node:retrieval_node:49 | [Retrieval Node] 主查询: Python 函数基础
2025-12-06 23:23:40.672 | INFO     | rag.index_manager:__init__:79 | 📁 索引管理器初始化: data/indexes
2025-12-06 23:23:40.673 | INFO     | rag.embeddings:get_embeddings:162 | 🔢 创建 Embedding 模型: Qwen/Qwen3-Embedding-8B
2025-12-06 23:23:40.773 | WARNING  | workflows.nodes.retrieval_node:retrieval_node:64 | [Retrieval Node] 索引不存在，返回空结果
2025-12-06 23:23:40.773 | INFO     | workflows.nodes.retrieval_node:retrieval_node:77 | [Retrieval Node] 检索到 0 个相关文档
2025-12-06 23:23:40.774 | INFO     | workflows.nodes.retrieval_node:retrieval_node:96 | [Retrieval Node] 最终检索到 0 个文档
2025-12-06 23:23:40.775 | INFO     | workflows.nodes.quiz_generator_node:quiz_gene

事件: {'planner': {'learning_plan': {'topic': 'Python 函数基础', 'objectives': ['能够使用 def 关键字定义一个简单的函数并调用它', '掌握如何传递不同类型的参数（位置参数、关键字参数、默认参数）', '理解函数返回值的作用并能够正确使用 return 语句'], 'key_points': ['函数的定义与调用', '参数传递（位置参数、关键字参数、默认参数）', '返回值与函数调用结果的处理', '作用域与变量', '函数嵌套与高阶函数'], 'difficulty': 'beginner', 'estimated_time': 120}, 'messages': [AIMessage(content='已为您制定学习计划：\n\n📚 **学习主题**: Python 函数基础\n\n🎯 **学习目标**:\n1. 能够使用 def 关键字定义一个简单的函数并调用它\n2. 掌握如何传递不同类型的参数（位置参数、关键字参数、默认参数）\n3. 理解函数返回值的作用并能够正确使用 return 语句\n\n💡 **关键知识点**:\n• 函数的定义与调用\n• 参数传递（位置参数、关键字参数、默认参数）\n• 返回值与函数调用结果的处理\n• 作用域与变量\n• 函数嵌套与高阶函数\n\n📊 **难度级别**: beginner\n⏱️ **预计时间**: 120 分钟\n', additional_kwargs={}, response_metadata={}, id='7b84cac2-4d0e-4649-9f8a-ffb71a8adc61')], 'current_step': 'planner', 'updated_at': '2025-12-06T23:23:40.669770'}}

事件: {'retrieval': {'retrieved_docs': [], 'messages': [{'role': 'assistant', 'content': '\n\n📄 已检索到 0 个相关文档，将用于生成学习内容和练习题。'}], 'current_step': 'retrieval', 'updated_at': '2025-12-06T23:23:40.774605'}}



2025-12-06 23:24:09.567 | INFO     | workflows.nodes.quiz_generator_node:quiz_generator_node:135 | [Quiz Generator Node] 成功生成 8 道练习题，总分 100 分


事件: {'quiz_generator': {'quiz': {'questions': [{'id': 'q1', 'type': 'multiple_choice', 'question': '以下哪个关键字用于在Python中定义一个函数？', 'options': ['def', 'function', 'create', 'define'], 'answer': 'A', 'explanation': '函数定义使用 `def` 关键字，这是Python中定义函数的标准语法。', 'points': 15}, {'id': 'q2', 'type': 'multiple_choice', 'question': '关于Python函数的参数传递，以下说法正确的是？', 'options': ['参数必须按位置传递', '关键字参数不能和位置参数混合使用', '默认参数必须在参数列表的末尾', '函数参数可以按位置、关键字或默认值传递'], 'answer': 'D', 'explanation': '在Python中，函数调用时，参数可以按位置传递（即按顺序）、按关键字传递（即指定参数名）、或者两者结合。默认参数是在定义函数时赋予的初始值，调用时可以省略。', 'points': 15}, {'id': 'q3', 'type': 'multiple_choice', 'question': '关于Python函数的返回值，以下说法正确的是？', 'options': ['函数无法返回值', '函数调用后可以通过变量接收返回值', '返回值只能是整数类型', '函数必须返回一个值'], 'answer': 'B', 'explanation': '返回值是函数执行后输出的结果，可以通过 `return` 语句返回。调用函数后，可以将返回值赋给变量进行处理。', 'points': 15}, {'id': 'q4', 'type': 'multiple_choice', 'question': '在Python中，以下关于作用域的说法正确的是？', 'options': ['函数内部定义的变量是全局变量', '函数内部无法访问外部变量', '函数内部定义的变量是局部变量', '全局变量在函数内部不可修改'], 'answer': 'C', 'explana

## 13. 完整的工作流测试函数

In [ ]:
def test_complete_workflow(topic: str):
    """测试完整的学习工作流"""
    
    thread_id = f"complete_test_{uuid.uuid4().hex[:8]}"
    
    print(f"\n{'='*60}")
    print(f"完整工作流测试: {topic}")
    print(f"Thread ID: {thread_id}")
    print(f"{'='*60}\n")
    
    # 1. 启动工作流
    print("1️⃣  启动工作流...")
    result = start_study_flow(user_question=topic, thread_id=thread_id)
    print("   ✅ 工作流启动成功\n")
    
    # 2. 显示学习计划
    if result.get('learning_plan'):
        plan = result['learning_plan']
        print("2️⃣  学习计划:")
        print(f"   主题: {plan.get('topic')}")
        print(f"   难度: {plan.get('difficulty')}")
        print(f"   目标数: {len(plan.get('objectives', []))}\n")
    
    # 3. 显示练习题
    if result.get('quiz'):
        quiz = result['quiz']
        print("3️⃣  练习题:")
        print(f"   题目数: {len(quiz.get('questions', []))}")
        print(f"   总分: {quiz.get('total_points')}\n")
    
    # 4. 模拟答题（全部答对）
    print("4️⃣  提交答案（全部答对）...")
    questions = quiz.get('questions', [])
    correct_answers = {q['id']: q['answer'] for q in questions}
    
    result = submit_answers(thread_id, correct_answers)
    print("   ✅ 答案提交成功\n")
    
    # 5. 显示评分
    print("5️⃣  评分结果:")
    print(f"   得分: {result.get('score', 0)} 分")
    print(f"   是否需要重试: {'是' if result.get('should_retry') else '否'}\n")
    
    # 6. 显示反馈
    if result.get('feedback'):
        print("6️⃣  反馈:")
        print(f"   {result['feedback'][:200]}...\n")
    
    print(f"{'='*60}")
    print("✅ 完整工作流测试完成！")
    print(f"{'='*60}\n")
    
    return thread_id

# 使用示例
# test_thread = test_complete_workflow("学习 JavaScript 的异步编程")

## 14. 总结

通过本 Notebook，你已经学习了：

✅ LangGraph 工作流的基本概念  
✅ 学习工作流的完整流程（规划→检索→出题→评分→反馈）  
✅ 工作流的启动与状态管理  
✅ 检查点（Checkpoint）机制与状态恢复  
✅ 人机交互（Human-in-the-Loop）  
✅ 重试机制（得分低于60分）  
✅ 流式输出工作流进度  
✅ 工作流历史记录查询

**下一步**: 学习第 4 阶段 - DeepAgents 深度研究（`stage_04_deep_research.ipynb`）